In [34]:
import pandas as pd

In [59]:
stats_df = pd.read_csv('../data/full_player_data_1991-2022.csv')

In [36]:
stats_df.head(5)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,0.476,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
1,Byron Scott,SG,29,LAL,82,82,32.1,6.1,12.8,0.477,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
2,Elden Campbell,PF,22,LAL,52,0,7.3,1.1,2.4,0.455,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
3,Irving Thomas,PF,25,LAL,26,0,4.2,0.7,1.9,0.340,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73
4,James Worthy,SF,29,LAL,78,74,38.6,9.2,18.7,0.492,...,0.0,0.0,Los Angeles Lakers,58,24,0.707,5.0,106.3,99.6,6.73


# Use the data for machine learning to predict MVP voting

## Extract Features from the data, split the data into training / testing sets

In [46]:
numeric_cols = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year','Pts Won', 'Pts Max', 'Share', 'W', 'L', 'W/L%', 'GB', 'PS/G','PA/G', 'SRS']
# Share = MVP Voting Share - the target of our supervised regression 
target = 'Share'

# Leave out the "Pts Max" and "Pts Won" columns, since these go into computing "Share" (the target feature)
target_related_features = ['Pts Won', 'Pts Max']

# Exclude columns that are highly related to other columns (ex. team losses are the complement of team wins (82 - wins) so W and L can be replaced by W/L%, 2PA can be computed by 2P and 2P%, etc.)
collinear_features = ['G', 'FGA', '3PA', '2PA', 'FTA', 'DRB', 'W','L', 'PS/G', 'PA/G' ]

# Filter the numeric columns to only extract our desired features
prediction_features = []
for col in numeric_cols: 
    if (col != target) and (col not in target_related_features) and (col not in collinear_features):
        prediction_features.append(col) 

prediction_features


['Age',
 'GS',
 'MP',
 'FG',
 'FG%',
 '3P',
 '3P%',
 '2P',
 '2P%',
 'eFG%',
 'FT',
 'FT%',
 'ORB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'Year',
 'W/L%',
 'GB',
 'SRS']

In [47]:
# Use the data from every NBA season 1991 -2022 as training data
training_df = stats_df[stats_df['Year'] < 2022]
# We want to predict the 2022 NBA MVP - use this as our testing data
testing_df = stats_df[stats_df['Year'] == 2022]

## Perform Data Preprocessing & Model Selection - implement feature scaling and dimensionality reduction, then train and fit various supervised regression models, perform hyperparameter tuning using cross-validation within the training set, and consider tradeoffs of different model types

In [48]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.svm import SVR
# Map models to their hyperparams for hyperparameter tuning
random_state= 1
models = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}
    },
    'Ridge': {
        'model': Ridge(),
        'params': {'ridge__alpha': [0.1, 1.0, 10.0]}
    },
    'RandomForestRegressor': {
        'model': RandomForestRegressor(random_state=random_state),
        'params': {'randomforestregressor__n_estimators': [50, 100, 200], 'randomforestregressor__max_depth': [3, 5, 7, None]}
    },
    'GradientBoostingRegressor': {
        'model': GradientBoostingRegressor(random_state=random_state),
        'params': {'gradientboostingregressor__n_estimators': [50, 100, 200], 'gradientboostingregressor__learning_rate': [0.01, 0.1, 0.2]}
    },
    'SVR': {
        'model': SVR(),
        'params': {'svr__C': [0.1, 1.0, 10.0], 'svr__epsilon': [0.01, 0.1, 0.2]}
    },
}


## Leverage SKLearn pipelining to streamline data pre-processing, model fitting, and hyperparameter tuning

In [49]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

X_train = training_df[prediction_features]
y_train = training_df[target]

# Map each model type to the pipeline w/ its optimal hyperparameters
best_pipelines = {}
# Map each model type to the best score obtained by optimal hyperparans
best_scores = {}

for model_name, config in models.items():
    pipeline = make_pipeline(
        # Standardize features by scaling mean to 0 and use unit variance (1)
        StandardScaler(),
        # Reduce the dimensionality to 15 features while preserving the most variance
        PCA(n_components=15),
        config['model']
    )
    
    # Conduct an exhaustive search to find the optimal param combinations
    # Perform 5-fold cross validation - for the data in the training set: pre-process the data w/ scaling and dimensionality reduction 
    # Use all available CPUs and neagtive MSE as our error metric
    grid_search = GridSearchCV(estimator=pipeline, param_grid=config['params'], cv=5, n_jobs=-1, scoring='neg_mean_squared_error')

    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)

    # Store the pipeline w/ best hyperparameters and scores 
    best_pipelines[model_name] = grid_search.best_estimator_
    # Convert back to positive MSE
    best_scores[model_name] = -grid_search.best_score_ 

    
print(f'Best Pipelines: {best_pipelines}')
print(f'Best Scores: {best_scores}')    

Best Pipelines: {'LinearRegression': Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=15)),
                ('linearregression', LinearRegression())]), 'Ridge': Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=15)), ('ridge', Ridge(alpha=10.0))]), 'RandomForestRegressor': Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=15)),
                ('randomforestregressor',
                 RandomForestRegressor(n_estimators=50, random_state=1))]), 'GradientBoostingRegressor': Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=15)),
                ('gradientboostingregressor',
                 GradientBoostingRegressor(n_estimators=50, random_state=1))]), 'SVR': Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=15)), ('svr', SVR(epsilon=0.01))])}
Best Scores: {'LinearR

In [50]:
from sklearn.pipeline import Pipeline
# Find the model with the lowest MSE - it is a SVR with epsilon = 0.01 and default params 
best_model_type : str = min(best_scores, key=best_scores.get)
best_pipeline : Pipeline = best_pipelines[best_model_type]
print(f'Best model type: {best_model_type}')
best_pipeline

Best model type: SVR


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=15)), ('svr', SVR(epsilon=0.01))])

## A SVR with epsilon=0.01 and default SKLearn params seems to be our most promising model. Re-train our SVR on the entire training set and employ various strategies for model evaluation. Consider if post-processing model predictions to better suit the context of predicting NBA MVP vote shares is appropriate and helpful to the model.  

In [51]:
best_pipeline.fit(X_train, y_train)

# Evaluate the model on the test dataset 
X_test = testing_df[prediction_features]
y_test = testing_df[target]
y_pred = best_pipeline.predict(X_test)

# Evaluate the model again with post-processing prediction 
# Since a player can't receive negative vote shares, we can assess our model performance with clipping as well 
import numpy as np
# Clip negative predictions to zero
y_pred_clipped = np.maximum(y_pred, 0)

mse_original = mean_squared_error(y_pred=y_pred, y_true=y_test)
print(f'Mean Squared Error (original) on test data: {mse_original}')

mse_clipped = mean_squared_error(y_pred=y_pred_clipped, y_true=y_test)
print(f'Mean Squared Error (clipped) on test data: {mse_clipped}')


Mean Squared Error (original) on test data: 0.00046203435714833906
Mean Squared Error (clipped) on test data: 0.00043430998844187394


In [53]:
# A small improving in MSE after clipping suggests that our model is robust, and the post-processing clipping can be kept to ensure model predictions are consistent with real MVP voting outcomes, since players cannot receive negative votes
y_pred = y_pred_clipped

# Create a series column containing our predictions 
predictions_df = pd.DataFrame(y_pred, columns=['predictions'], index=testing_df.index)
predictions_df

,predictions
648,0.000000
649,0.000000
650,0.000000
651,0.001119
652,0.000000
...,...
12508,0.002877
12509,0.002814
12510,0.005306
12511,0.000000


## Store our model's predictions in a date frame, compare the predicted MVP voting ranks of players to their actual MVP voting ranks

In [54]:
mvp_votes_with_preds_2022_df = pd.concat(
    [testing_df[['Player','Share']], predictions_df], axis=1
)
# Rename the columns for more clarity 
mvp_votes_with_preds_2022_df['Predicted Share'] = mvp_votes_with_preds_2022_df['predictions']
del mvp_votes_with_preds_2022_df['predictions']

mvp_votes_with_preds_2022_df['Actual Share'] = mvp_votes_with_preds_2022_df['Share'] 
del mvp_votes_with_preds_2022_df['Share']
mvp_votes_with_preds_2022_df

,Player,Predicted Share,Actual Share
648,Aaron Gordon,0.000000,0.0
649,Austin Rivers,0.000000,0.0
650,Bol Bol,0.000000,0.0
651,Bones Hyland,0.001119,0.0
652,Bryn Forbes,0.000000,0.0
...,...,...,...
12508,Micah Potter,0.002877,0.0
12509,Rodney McGruder,0.002814,0.0
12510,Saben Lee,0.005306,0.0
12511,Saddiq Bey,0.000000,0.0


In [55]:
# Let's see who our algorithm predicted to win MVP
mvp_votes_with_preds_2022_df.sort_values('Actual Share', ascending=False).head(10)

,Player,Predicted Share,Actual Share
663,Nikola Jokić,0.688200,0.875
837,Joel Embiid,0.435592,0.706
11678,Giannis Antetokounmpo,0.489777,0.595
907,Devin Booker,0.052345,0.216
11469,Luka Dončić,0.188067,0.146
1179,Jayson Tatum,0.080816,0.043
12226,Ja Morant,0.227646,0.010
6398,Stephen Curry,0.065972,0.004
905,Chris Paul,0.095576,0.002
8241,LeBron James,0.000000,0.001


In [56]:
# Sort our DF by actual and predicted shares in descending order
mvp_votes_with_preds_2022_df = mvp_votes_with_preds_2022_df.sort_values(['Actual Share', 'Predicted Share'], ascending=False)

# Assign ranks based on actual shares, considering ties for players who received 0% of MVP voting shares
mvp_votes_with_preds_2022_df['Actual Rank'] = mvp_votes_with_preds_2022_df['Actual Share'].rank(method='min', ascending=False).astype(int)

mvp_votes_with_preds_2022_df['Predicted Rank'] = mvp_votes_with_preds_2022_df['Predicted Share'].rank(method='min', ascending=False).astype(int)

mvp_votes_with_preds_2022_df.head(30)

,Player,Predicted Share,Actual Share,Actual Rank,Predicted Rank
663,Nikola Jokić,0.688200,0.875,1,1
837,Joel Embiid,0.435592,0.706,2,3
11678,Giannis Antetokounmpo,0.489777,0.595,3,2
907,Devin Booker,0.052345,0.216,4,11
11469,Luka Dončić,0.188067,0.146,5,5
1179,Jayson Tatum,0.080816,0.043,6,9
12226,Ja Morant,0.227646,0.010,7,4
6398,Stephen Curry,0.065972,0.004,8,10
905,Chris Paul,0.095576,0.002,9,8
6185,Kevin Durant,0.109532,0.001,10,7


In [101]:
# There seem to be completely "random" players sneaking into the top 30 in votes (ex. Jaden Springer, Craig Sword, Derrick Walton )
# Examing their 2022 stats on basketball reference.com, they all have one thing in common: a category in which they are shooting 100% 
stats_df_2022 = stats_df[stats_df['Year'] == 2022]


shooting_cols = ['FG%', 'eFG%', '2P%', '3P%', 'FT%']
dfs = []

# For each shooting column, filterthe df for players with shooting percentage >= 0.99 for that column
for col in shooting_cols:
    elite_shooters_df = stats_df_2022[stats_df_2022[col] >= 0.99]
    dfs.append(elite_shooters_df)

# Concatenate all filtered dfs into a single df
filtered_df = pd.concat(dfs, ignore_index=True)

outliers = filtered_df['Player'].unique()

top_30_vote_recipients = mvp_votes_with_preds_2022_df['Player'][:30].to_list()


outlier_vote_recipients = [player for player in outliers if player in top_30_vote_recipients]

outlier_vote_recipients

['Jaden Springer', 'Craig Sword', 'Derrick Walton']

## Feature Selection- perform a PCA analysis and correlation analysis for insights into which features are most impactful. Investigate the outcome of PCA in our SVR pipeline - assess which features the PCA selected as most impactful on NBA MVP voting share variance. Examine which features are directly related to our target variable using linear correlation

## 

In [ ]:
from sklearn.decomposition import PCA
from typing import Dict, List
import pandas as pd
import numpy as np

pca: PCA = best_pipeline.named_steps['pca']
n_components = pca.n_components_

# Store the explained variance ratios for each principal component
evr_list: List[float] = pca.explained_variance_ratio_.tolist()

# Rank the features in descending order based on their impact on the variance in NBA MVP voting shares 
# Map each feature to its cumulative impact 
feature_impacts : Dict[str, float] = {
    feature_name : 0.0 for feature_name in prediction_features
}

# Calculate the cumulative impact for each feature
for component_idx in range(n_components):
    component_evr = evr_list[component_idx]
    for feature_idx, feature_name in enumerate(prediction_features):
        # For each feature in each component, add the absolute value of the feature's loading (coefficient) weighted by the explained variance ratio of the component to the cumulative impact of the feature.
        feature_loading_impact = np.abs(pca.components_[component_idx, feature_idx]) 
        weighted_impact =  feature_loading_impact * component_evr
        feature_impacts[feature_name] += weighted_impact

# Store the weighted impact data in our DF
features_df = pd.DataFrame({
    'Feature Name': list(feature_impacts.keys()),
    'Cumulative PCA Impact': list(feature_impacts.values())
},  )
 
 # Sort the DF by cumulative PCA impact in descending order 
features_df = features_df.sort_values(by='Cumulative PCA Impact', ascending=False).reset_index(drop=True)
features_df


,Feature Name,Cumulative PCA Impact
0,AST,0.146522
1,TRB,0.144178
2,DRB,0.142143
3,2PA,0.140375
4,PF,0.139966
5,FTA,0.138686
6,FGA,0.138577
7,STL,0.138216
8,2P,0.137779
9,TOV,0.137474


In [ ]:
# Select top features based on PCA cumulative impact
top_pca_features = features_df['Feature Name'].head(10).tolist()
top_pca_features

['AST', 'TRB', 'DRB', '2PA', 'PF', 'FTA', 'FGA', 'STL', '2P', 'TOV']

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.tree import DecisionTreeRegressor

# Create a pipeline w/ standardization and recursive feature elimination, using a Random Forest regressor with the best hyperparamaters we found
pipeline = make_pipeline(
    StandardScaler(),
    RFE(estimator = DecisionTreeRegressor(), n_features_to_select=15),
)

# Perform repeated k-fold cross-validation and evaluate model performance
kfold_cv : RepeatedKFold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=1)
mse_scores = cross_val_score(pipeline, X_train, y_train, scoring='neg_mean_squared_error', cv=kfold_cv, n_jobs=-1)

# Convert scores to positive valuesc
mse_scores = -mse_scores

# Calculate the average MAE score
print(f"Average Cross-Validation MSE:{np.mean(mse_scores)}")

Average Cross-Validation MSE:0.0026735602472424897


In [ ]:


from sklearn.feature_selection import RFECV
# Create an RFE object with your desired estimator and number of features to select
rfe = RFECV(estimator=DecisionTreeRegressor(),  cv=5)

# Fit RFE on the training data
rfe.fit(X_train, y_train)

# Get the indices of the selected features
selected_features_indices = rfe.support_
rfe_selected_cols = X_train.columns[selected_features_indices]

rfe_selected_cols

Index(['G', 'MP', 'FG', 'FG%', '3P%', '2P', '2P%', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L',
       'W/L%', 'PA/G', 'SRS'],
      dtype='object')

In [ ]:
#  Observe which numeric cols are most closely associated with MVP vote share  
numeric_cols_df = training_df.select_dtypes(include=['number'])

# Calculate the correlation matrix for the DF - examine which features have the strongest linear relationships with MVP voting share 
corr_matrix = numeric_cols_df.corr()

# Select top features based on correlation with MVP voting share
corr_features = corr_matrix['Share'].abs().sort_values(ascending=False).index.to_list()

# Filter out features that are not used for model training (Share, PTS Won, PTS MAX), since these features directly impact MVP voting shares (our target variable) 
corr_features = [feature for feature in corr_features if feature in prediction_features]

# Choose the 15 most highly-correlated features with MVP voting share 
top_corr_features = corr_features[:15]
top_corr_features

['FTA',
 'FT',
 'PTS',
 'FG',
 '2P',
 '2PA',
 'FGA',
 'TOV',
 'DRB',
 'AST',
 'TRB',
 'STL',
 'GS',
 'MP',
 'BLK']

In [ ]:
# Combine both sets of features
final_features = list(set(top_pca_features + ))

# Train the model using these combined features
X_train_final = X_train[final_features]
X_test_final = X_test[final_features]

# Example model training with combined features
best_pipeline.fit(X_train_final, y_train)

# Evaluate the model
y_pred_final = best_pipeline.predict(X_test_final)

y_pred_final_clipped = np.maximum(y_pred_final, 0)

mse_final_original = mean_squared_error(y_pred=y_pred_final, y_true=y_test)
print(f'Mean Squared Error (original) on final test data: {mse_final_original}')

mse_final_clipped = mean_squared_error(y_pred=y_pred_final_clipped, y_true=y_test)
print(f'Mean Squared Error (clipped) on final test data: {mse_final_clipped}')

# Mean Squared Error (original) on test data: 0.0007010428821412067
# Mean Squared Error (clipped) on test data: 0.000687832951861502

Mean Squared Error (original) on final test data: 0.0007568611665452203
Mean Squared Error (clipped) on final test data: 0.0007274221704793133


## Interpreting the final error metrics after feature selection 
- Model Performance after original feature selection: (after PCA) 
    - Mean Squared Error (original) on test data: 0.0007010428821412067
    - Mean Squared Error (clipped) on test data: 0.000687832951861502
- Final feature selection (after PCA output anaylsis and correlation matrix analysis)
    - Mean Squared Error (original) on final test data: 0.0007568611665452203
    - Mean Squared Error (clipped) on final test data: 0.000727422170479313   
- High-dimensional datasets such as this can be prone to overfitting. Although the final feautre selection based on PCA and corrrelation matrix assessment could indicate 
## Addressing the bias-variance tradeoff